In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

In [7]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

In [8]:
import time

import warnings
warnings.filterwarnings('ignore')

In [9]:
from datetime import datetime

current_time = datetime.now()

print(current_time.strftime("%Y%m%d%H%M%S"))

20240820085755


In [10]:
list_url = 'https://map.naver.com/p/search/충주시%20식당?c=10.00,0,0,0,dh'
starRatingReviewList = []
starRatingList = []
reviewList = []
starRatingResturantList = []
restaurantList = []
reviewDataList_contain_star = []
reviewDataList_without_star = []
restaurantsReviewList_star = []
restaurantsReviewList_nonstar = []
breakPoint = False;
s = Service()
options = webdriver.ChromeOptions()
wd = webdriver.Chrome(service=s,options=options)
wd.get(list_url)
time.sleep(4)

# 페이지 다운
def page_down(num):
    body = wd.find_element(By.TAG_NAME, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)



for i in range(1,4):
    
    #페이지 로딩 대기
    time.sleep(2)
    wd.switch_to.default_content()
    wd.switch_to.frame('searchIframe')
    time.sleep(4)
    page_down(50)
    time.sleep(2)

    isStartPoint = False
    for j in range(1, 50):
        try:
            # 리뷰 데이터 리스트 초기화
            reviewDataList_contain_star = []
            reviewDataList_without_star = []
            body = wd.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li:nth-child(%d) > div.CHC5F > a > div > div > span.TYaxT' %j)
            body.click()
            print("버튼 클릭")
            time.sleep(2)
            # 클릭하게 되면 div sc-1wsjitl OWZjJ가 개발자 도구화면에 생김.
            wd.switch_to.default_content()
            wd.switch_to.frame('entryIframe')
            
            
            starRating = wd.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]').text
            print(starRating)
            # 정보 수집 끝이므로, 종료.
            if len(starRatingReviewList) >= 80 and len(reviewList) >= 20:
                breakPoint = True
                break

            if '별점' in starRating:
                #별점이 있는 리뷰
                review = wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
                print(review)
                review_num = int(re.sub(r'[^0-9]', '', review))
                #별점
                starRatingInsertComma = re.sub(r'[^0-9]', '', starRating)
                starRating = float(starRatingInsertComma[:1] + "." + starRatingInsertComma[1:])      

                # 방문자 리뷰가 100 이하면 컨티뉴
                if review_num < 100 or len(starRatingReviewList) >= 80:
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                # 방문자 리뷰의 리스트가 80이상이면, 더이상 추가 x
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                
                # 별점이 존재하는 방문자리뷰 리스트에 더한다.
                starRatingReviewList.append(review_num)
                starRatingRestaurantName = wd.find_element(By.XPATH, '//*[@id="_title"]/div/span[1]').text
                print(starRatingRestaurantName)
                starRatingResturantList.append(starRatingRestaurantName)
                starRatingList.append(starRating)

                isStartPoint = True # 더보기를 눌렀을 때, 별점이 있는 리뷰 데이터를 분류하기 위해서 flag 설정 => True 별점이 있다. false 별점이 없다.
            else:
                # 별점이 없는 리뷰
                review = wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
                print(review+'#')
                review_num = int(re.sub(r'[^0-9]', '', review))

                # 방문자 리뷰가 100 이하면 컨티뉴
                if review_num < 100 or len(reviewList) :
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                    # 별점이 존재하지 않는 방문자 리뷰의 리스트가 20이상이면, 더이상 추가 x
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문

                # 별점이 존재하지 않는 방문자 리뷰에 더한다.
                reviewList.append(review_num)
                restaurantName = wd.find_element(By.XPATH, '//*[@id="_title"]/div/span[1]').text
                print(restaurantName)
                restaurantList.append(restaurantName)

                isStartPoint = False # 더보기를 눌렀을 때, 별점이 없는 리뷰 데이터를 분류하기 위해서 flag 설정

            time.sleep(2)
            # 리뷰 버튼 클릭
            wd.find_element(By.CSS_SELECTOR, f'a[href*="review?entry"]').click()

            cnt = 0
            # 리뷰 페이지를 내리면서 더보기로 갱신을 반복
            while True:
                try:
                    cnt += 1
                    time.sleep(1) 
                    wd.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                    time.sleep(3)
                    wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a').click() 
                    time.sleep(3) 
                    wd.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                    time.sleep(1)
                    print(cnt)
                    # 리뷰 데이터가 워낙 많으므로 100개만 수집
                    if cnt == 10:
                        print("리뷰 데이터 100개 수집")
                        break
                except NoSuchElementException: 
                    print('-더보기 버튼 모두 클릭 완료-')
                    break
            
            reviewData = wd.find_elements(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.place_section_content > ul > li')
            
            
            for review in reviewData:
                try:
                    # 리뷰 작성자
                    review_writer = review.find_element(By.CLASS_NAME, 'pui__uslU0d').text
                    print(review_writer)
                    
                    #  리뷰 데이터, 데이터가 없을시 NoSuchElementException 에러
                    description = review.find_element(By.CLASS_NAME, 'pui__xtsQN-').text
                        
                    processed_reviewData = (review_writer, description)

                    if isStartPoint == True:
                        reviewDataList_contain_star.append(processed_reviewData)
                    else :
                        reviewDataList_without_star.append(processed_reviewData)
                except NoSuchElementException:
                    # 사진만 올라온 경우
                    description = ''
                    processed_reviewData = (review_writer, description)           
                    if isStartPoint == True:
                        reviewDataList_contain_star.append(processed_reviewData)
                    else :
                        reviewDataList_without_star.append(processed_reviewData)

            print('a')
            #하나의 모아진 가게 데이터 리뷰를 전체 가게 리뷰 리스트로 넣는다.
            restaurantsReviewList_star.append(reviewDataList_contain_star)
            restaurantsReviewList_nonstar.append(reviewDataList_without_star)
            print('b')
            wd.switch_to.default_content()
            wd.switch_to.frame('searchIframe')
            print('c')
        except:
            wd.switch_to.default_content()
            wd.switch_to.frame('searchIframe')
            print("none")

    # 정보 수집 끝일시, 종료
    if breakPoint == True:
        break
    # 페이지 이동
    wd.find_element(By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(%d)' %(i+2)).click()
    
print("별점이 없는 방문자 리뷰 리스트")
print(reviewList)
print(restaurantList)
print("별점이 존재하는 방문자 리뷰 리스트")
print(starRatingReviewList)
print(starRatingResturantList)

버튼 클릭
별점
4.42
방문자 리뷰 429
본가할매숨두부 꿩요리
1
2
3
4
5
6
7
8
9
10
리뷰 데이터 100개 수집
lucky inz
sports7210
mea****
SoonH
맛집일상
sag****
SKY7CEO
JooL
유나짱맛집
한감자님
뽀뚱뚱
다사랑696
수예태
땡잡은놈
하늘이5758
단미1885
DesignerH
qqg****
박하사탕288
찬슬맘36
아이스크림9646
mih****
jhc****
oxwhite
삐삐7649
jsl****
oxwhite
au****
보령맛집 우미장
역사지기
K갤러리
Baegopayoyo
몽중인121
arm****
dsub387
lovekj0202
lsrsh
kjo****
지공41
kimmh0830
산여울Tv
123****
보성키위
풀벌레77
님임님7272
tnr****
황후015732
oxwhite
redstone6707
costa810
Somiii93
완소여우
여행하고 싶은 사람
hyl****
미사티아라98
미소맘370
쭈야쭈야88
dsub387
황후015732
to****
상 범
sjy
동혁30
룡이13
라이어1탄
더레드맨74
나나97
diana2
man****
레몬에이드4625
참치맘68
참치맘68
WD9
삶을 즐기는 여자
als****
queenka427
yuu****
조용한친구야
하늘보기1175
조미료애비
풀벌레77
통반장
단미1885
야마77
abc****
찐이야5397
로이랜드
덕운 하형사
달개비사랑
양남처사
하리보젤리67
꼬꼬마얌
limonada
KIM J 0
hso****
써니4585
요가녀
123hjp15
올라프3428
lj****
djw****
달아이11
akf****
용용829
뽐뽐630
whr****
유클리드15
mouseluck
whtlily1112
pup****
a
b
c
버튼 클릭
별점
4.2
방문자 리뷰 258
수안보 명화식당 꿩요리
1
2
3
4
5
6
7
8
9
10
리뷰 데이터 100개 수집
도형580
도도별83
호야도리3
aa****
용용용11003
블루벨95
드저드

In [11]:
print("별점이 없는 방문자 리뷰 갯수")
print(reviewList)
print("별점이 없는 가게 이름")
print(restaurantList)
print("별점이 없는 가게 갯수")
print(len(reviewList))
print(len(restaurantList))
print("별점이 존재하는 방문자 리뷰 갯수")
print(starRatingReviewList)
print("별점이 존재하는 가게 이름")
print(starRatingResturantList)
print("별점이 존재하는 가게 별점들")
print(starRatingList)
print("별점이 존재하는 가게 갯수")
print(len(starRatingList))
print(len(starRatingReviewList))
print(len(starRatingResturantList))


별점이 없는 방문자 리뷰 갯수
[579]
별점이 없는 가게 이름
['돈이 조아']
별점이 없는 가게 갯수
1
1
별점이 존재하는 방문자 리뷰 갯수
[429, 258, 1142, 2342, 2955, 770, 2273, 1478, 300, 1247, 797, 584, 673, 873, 287, 1643, 776, 1212, 1031, 1711, 575, 524, 1323, 1429, 294, 341, 614, 270, 876, 1562, 558, 110, 561, 343, 492, 302, 434, 606, 985, 690, 867, 378, 138, 952, 371, 137, 304, 1068, 212, 342, 210, 1024, 234, 900, 653, 400, 464, 829, 281, 729, 507, 246, 671, 4128, 202, 217, 314, 570, 198, 412, 399, 338, 430, 319, 200, 1512, 571, 1382, 495, 632]
별점이 존재하는 가게 이름
['본가할매숨두부 꿩요리', '수안보 명화식당 꿩요리', '열혈청춘 본점', '원조중앙탑막국수 충주점', '중앙탑메밀마당', '숲속장수촌', '중앙탑막국수', '후라토식당 충주점', '이도희한우생갈비 충주본점', '갈마가든', '청진궁중면옥 충주점', '상촌식당', '예반72', '함경옥 사과냉면 만두', '대장군', '충주홍두깨칼국수보쌈 원조본점', '향나무식당', '중앙탑뚱감자막국수', '민들레', '오뚜기분식', '카츠집', '삼정면옥', '이씨식당 충주점', '중앙탑면막국수', '시내짬뽕', '감나무집', '미사리밀빛초계국수 충주점', '벨라루나', '별난찜', '탄금대왕갈비탕', '고추장송어', '우하하', '그린가든', '풍년식당', '강금자손수제비', '플랩잭팬트리 충주점', '밍거스 바베큐', '국수타령', '신양자강', '방가삼대 얼큰한 뼈해장국', '무드인소울', '중앙탑막국수안림점', '산들애', '충주한우프라자', '조리터명가', '

In [16]:
print(restaurantsReviewList_star)

[[('lucky inz', '산채정식 2인 시켜서 먹었는데 나물 종류가 많고 다 만드신거더라구요 된장찌개가 집된장이라 약간 짯지만 구수하니 맛있어요 더덕구이랑 생선구이까지 나와서 푸짐해요~ 다만 음식 간이 전반적으로 센편이네요 그래도 맛있게 잘먹었습니다'), ('sports7210', '꿩요리는 처음인데 인상깊게 먹고 옵니다\n꿩육회도 쫄깃하고 잡내없고 불고기 샤브샤브까지 맛있게 먹고 왔어요 부모님 모시고가기 좋네요, 산채정싱 상 받으셨다던데 그래수 그런지 밑반찬들이 정갈하고 맛있었습니다.'), ('mea****', '여름 몸보신으로 겸사겸사 꿩요리 먹으러 왔어요 요즘 날씨가 꿉꿉했는데 맑은 탕이라 개운하고 코스요리라 양도 많고 여러가지로 즐길 수 있어서 너무 잘먹었는데 주변에 2차 갈때가 없어서 좀 늦게까지 먹었는데 사장님 친절하시고 잘 먹고 왔습니다~ 반찬들도 다 맛있었어요'), ('SoonH', '칭국장도 맛있고 반찬이 맛있어요~\n꿩요리도있고 다양한 메뉴가 많네요.가성비 좋아요.'), ('맛집일상', '꿩샤브샤브 담백하고 맛있네요 산채정식 상도 받은 집이라던데 밑반찬도 엄청 나물들 잘 무쳐서 나오더라구요~ 부모님들 모시고 가기 좋은 집 같고 사장님이 엄청 친절하세요 인신듀 좋으신데~ 맛있게 잘 먹고 갑니다~!!'), ('sag****', '꿩요리는 처음인데 사장님이 친절하게 먹는법도 알려주시고 맛있게 먹었어요 보니까 상도 받고 티비도 나온 집이더라구요 밑반찬도 정갈해서 잘 먹고 왔습니다'), ('SKY7CEO', '코스로 주문했는데 이렇게 다양하게 많이 나올줄은 몰랐어요 심지어 다 찍지도 못했는데 메뉴가 다 정성이 느껴짐, 그래도 관광지고 같은 메뉴들이 몰려있는거리라 좀 걱정하고 갔는데 밑반찬도 엄청 잘 나오는데 손수 다 하시는 것 같고 양념이 대체적으로 쎈느낌은 아니고 진짜 본연 맛 느낄 수 있는거랑 거부감 없이 먹을 수 있도록 메뉴구성이 좋은 것 같음 샤브샤브 국물에 마지막에 칼국수까지 해먹으면 진짜 배 터집니다 ㅎㅎ 만두도 윤기가 엄청나서 그민먹

In [13]:
print(len(restaurantsReviewList_nonstar))
print(len(restaurantsReviewList_star))

72
72


In [23]:
result_star = []
result_nonstar= []
for i in range(0, 72):
    if len(result_nonstar) == 20:
        break
    if restaurantsReviewList_nonstar[i] == []:
        continue
    result_nonstar.append(restaurantsReviewList_nonstar[i])
for i in range(0, 72):
    if len(result_star) == 80:
        break
    if restaurantsReviewList_star[i] == []:
        continue
    result_star.append(restaurantsReviewList_star[i])

result_nonstar.append(restaurantsReviewList_nonstar[0])
result_star.append(restaurantsReviewList_star[1])
result_star.append(restaurantsReviewList_star[2])
result_star.append(restaurantsReviewList_star[3])

In [24]:
print(len(result_star))
print(len(result_nonstar))

74
2


In [25]:
import csv
csv_file_path = 'output.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    for sublist in restaurantsReviewList_star:
        # Write the header row for each sublist
        csv_writer.writerow(['Author', 'Review'])
        
        # Write data from the current sublist
        csv_writer.writerows(sublist)
        
        # Write an empty row to separate the data between sublists
        csv_writer.writerow([])

In [26]:
import pandas as pd
# restaurantsReviewList_star, restaurantsReviewList_nonstar에는 별점이 있는 가게의 리뷰데이터, 없는 가게의 리뷰데이터를 분리하여 순서대로 배치했습니다. 

df = pd.DataFrame({'가게 이름': restaurantList, '리뷰 수': reviewList, '리뷰 데이터': result_nonstar})
df

ValueError: All arrays must be of the same length

In [ ]:
print(len(starRatingResturantList))
print(len(starRatingReviewList))
print(len(starRatingList))
print(len(result_star))

In [ ]:
# restaurantsReviewList_star, restaurantsReviewList_nonstar에는 별점이 있는 가게의 리뷰데이터, 없는 가게의 리뷰데이터를 분리하여 순서대로 배치했습니다. 
df2 = pd.DataFrame({'가게 이름': starRatingResturantList, '리뷰 수': starRatingReviewList, '별점': starRatingList, '리뷰 데이터' : result_star})
df2

In [ ]:
# 저장
df.to_csv('restaurants_nonstar.csv', sep='\t', encoding='utf-8')
df2.to_csv('restaurants_star.csv', sep='\t', encoding='utf-8')


In [ ]:
# 별점없는 리뷰 데이터들 라벨링
for reivewes in df['리뷰 데이터']:
    for data in reivewes:
        
        print(data)


In [ ]:
# 데이터 라벨링하는데 너무 많은 데이터이기에, 이미 학습된 긍정 분석, 부정 분석을 하는 라이브러리를 가져와 예측을 하는 것으로 대신 라벨링
from transformers import pipeline




In [ ]:
sentiment_analyzer = pipeline('sentiment-analysis', model='beomi/kcbert-base')



In [ ]:


# Load pre-trained sentiment analysis model for Korean
sentiment_analyzer = pipeline('sentiment-analysis', model='beomi/kcbert-base')

column = []
for reivewes in df['리뷰 데이터']:
    temp_list = []
    for data in reivewes:
        sentiment_result = sentiment_analyzer(data[1], max_length=512, truncation=True)

        predicted_sentiment = sentiment_result[0]['label']

        if predicted_sentiment == 'LABEL_0':
            #부정
            temp_list.append(0)
        else:
            #긍정
            temp_list.append(1)
    column.append(temp_list)
df['긍정 부정'] = column
df
        


In [ ]:
print(len(df['긍정 부정'][3]))
print(len(df['리뷰 데이터'][3]))

In [ ]:
column2 = []
for reivewes in df2['리뷰 데이터']:
    temp_list = []
    for data in reivewes:
        sentiment_result = sentiment_analyzer(data[1], max_length=512, truncation=True)

        predicted_sentiment = sentiment_result[0]['label']

        if predicted_sentiment == 'LABEL_0':
            #부정
            temp_list.append(0)
        else:
            #긍정
            temp_list.append(1)
    column2.append(temp_list)
df2['긍정 부정'] = column2
df2

In [ ]:

DataFrame_contain_list = []

for data in df['리뷰 데이터']:
    proccessd_data = []
    for element in data:
        proccessd_data.append(element[1])
    
    temp_df = pd.DataFrame({'리뷰': proccessd_data})

    DataFrame_contain_list.append(temp_df)
   
    

       

In [ ]:
DataFrame_contain_list[0]


In [ ]:
for index, value in enumerate(df['긍정 부정']):
    temp_df = DataFrame_contain_list[index]
    temp_df['긍정 부정'] = value
    DataFrame_contain_list[index] = temp_df

confirm_df = DataFrame_contain_list[0]


In [ ]:
# 1인경우
confirm_df[confirm_df['긍정 부정'] == 1]

In [ ]:
# 0인경우
confirm_df[confirm_df['긍정 부정'] == 0]

In [ ]:
# df2 전처리

DataFrame_contain_list2 = []

for data in df2['리뷰 데이터']:
    proccessd_data2 = []
    for element in data:
        proccessd_data2.append(element[1])
    
    temp_df2 = pd.DataFrame({'리뷰': proccessd_data2})

    DataFrame_contain_list2.append(temp_df2)
   
    

In [ ]:
DataFrame_contain_list2[0]

In [ ]:
for index, value in enumerate(df2['긍정 부정']):
    temp_df2 = DataFrame_contain_list2[index]
    temp_df2['긍정 부정'] = value
    DataFrame_contain_list2[index] = temp_df2

confirm_df2 = DataFrame_contain_list2[0]

In [ ]:
# 1인경우
confirm_df2[confirm_df2['긍정 부정'] == 1]

In [ ]:
# 0인경우
confirm_df2[confirm_df2['긍정 부정'] == 0]

In [ ]:
3. 각 식당의 리뷰 데이터에 대해 감성분석을 수행하여 전체 리뷰수에 대한 긍정 리뷰수의 비율을 구하시오. 긍정 리뷰수 비율을 포함한 데이터프레임을 만드시오. 
   - 별점이 있는 데이터 : 식당명, 위치(시), 별점, 리뷰수, 긍정리뷰비율
   - 별점이 없는 데이터 : 식당명, 위치(시), 리뷰수, 긍정리뷰비율


In [ ]:
# 
DataFrame_contain_list[0].info()

In [ ]:
print(DataFrame_contain_list[0]['긍정 부정'].value_counts())
# 긍정이 35 부정이 75개다.

In [ ]:
for index, value in enumerate(DataFrame_contain_list):
    DataFrame_contain_list[index]['리뷰'] = DataFrame_contain_list[index]['리뷰'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))


for index, value in enumerate(DataFrame_contain_list2):
    DataFrame_contain_list2[index]['리뷰'] = DataFrame_contain_list2[index]['리뷰'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))


In [ ]:
print(len(DataFrame_contain_list))
print(len(DataFrame_contain_list2))

In [ ]:
from konlpy.tag import Okt

okt = Okt()

In [ ]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 별점없는 데이터 훈련세트와 테스트 세트로 분리, 별점있는 데이터 훈련세트와 테스트 세트로 분리 
df_contain_train_list = []
df_contain_test_list = [] 
df2_contain_train_list = []
df2_contain_test_list = []

for index, value in enumerate(DataFrame_contain_list):
    DataFrame_contain_list_train, DataFrame_contain_list_test = train_test_split(DataFrame_contain_list[index], test_size=0.2, random_state=42)

    df_contain_train_list.append(DataFrame_contain_list_train)
    df_contain_test_list.append(DataFrame_contain_list_test) 

for index, value in enumerate(DataFrame_contain_list2):
    DataFrame_contain_list2_train, DataFrame_contain_list2_test = train_test_split(DataFrame_contain_list2[index], test_size=0.2, random_state=42)

    df2_contain_train_list.append(DataFrame_contain_list2_train)
    df2_contain_test_list.append(DataFrame_contain_list2_test)



In [ ]:
print(len(df_contain_train_list))
print(len(df2_contain_train_list))

In [ ]:
collected_train_List = []
collected_train_List2 = []
collected_train_emotion_List = []
collected_train_emotion_List2 = []
for df_train in df_contain_train_list:
    for element in df_train['리뷰']:
    
        collected_train_List.append(element)
    for element in df_train['긍정 부정']:
        collected_train_emotion_List.append(element)
        
for df2_train in df2_contain_train_list:
    for element in df2_train['리뷰']:
        
        collected_train_List2.append(element)
    for element in df2_train['긍정 부정']:
        collected_train_emotion_List2.append(element)

print(len(collected_train_List))
print(len(collected_train_List2))
print(len(collected_train_emotion_List))
print(len(collected_train_emotion_List2))

fin_train_df = pd.DataFrame({'리뷰': collected_train_List, '긍정 부정': collected_train_emotion_List})
fin_train_df2 = pd.DataFrame({'리뷰': collected_train_List2, '긍정 부정': collected_train_emotion_List2})


In [ ]:
collected_test_emotion_List = []
collected_test_emotion_List2 = []
collected_test_List = []
collected_test_List2 = []

for df_test in df_contain_test_list:
    for element in df_test['긍정 부정']:        
        collected_test_emotion_List.append(element)
    for element in df_test['리뷰']:
        collected_test_List.append(element)
        
for df2_test in df2_contain_test_list:
    for element in df2_test['긍정 부정']:     
        collected_test_emotion_List2.append(element)
    for element in df2_test['리뷰']:
        collected_test_List2.append(element)

print(len(collected_test_List))
print(len(collected_test_List2))
print(len(collected_test_emotion_List))
print(len(collected_test_emotion_List2))

fin_test_df = pd.DataFrame({'리뷰' : collected_test_List, '긍정 부정': collected_test_emotion_List})
fin_test_df2 = pd.DataFrame({'리뷰' : collected_test_List2, '긍정 부정': collected_test_emotion_List2})

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(fin_train_df['리뷰'])
non_star_train_tfidf = tfidf.transform(fin_train_df['리뷰'])


In [ ]:
print(non_star_train_tfidf)


In [ ]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [ ]:
SA_lr.fit(non_star_train_tfidf, fin_train_df['긍정 부정'])

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [ ]:
SA_lr_grid_cv.fit(non_star_train_tfidf, fin_train_df['긍정 부정'])

In [ ]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

In [ ]:
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [ ]:
non_star_test_tfidf = tfidf.transform(fin_test_df['리뷰'])

In [ ]:
test_predict = SA_lr_best.predict(non_star_test_tfidf)

In [ ]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(fin_test_df['긍정 부정'], test_predict), 3))

In [ ]:
# DataFrame_contain_list = []

for restarunt_df in DataFrame_contain_list:
    print(restarunt_df)